# Web Scraping Student
## For: Michel Souza Santana
> Date start: 01/02/2022

* The purpose of this is to record the process of learning the WebScraping technique and make it available to anyone who wants to make use of it for their own development.

#### Instalando a biblioteca BeautifulSoup

In [110]:
# !pip install bs4

#### Importando as bibliotecas

In [111]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests
import pandas as pd
import numpy as np

#### Coletando dados on-line via URL

> Fontes dos dados coletados
* https://www.adimimoveis.com.br/imoveis/a-venda 

##### OBS: A parte final desse código salva todo conteúdo em um arquivo html, assim sendo não será necessário rodar esse código a cada vêz para carregar os dados pois este será carregado a partirdo arquivo salvo polpando tempo em casos de consultas prolongadas. 

#### Por meio da url estarei capturando a quantidade de páginas existente no site para dai então percorrer uma a uma coletando os valores de meu interesse.

In [112]:
url="https://www.adimimoveis.com.br/imoveis/a-venda"
lista_paginacao = []
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
html = urlopen(req)
bs = BeautifulSoup(html.read(), 'html.parser')
lista_paginacao.append(bs.find_all('div', {'class':'pagination-cell hidden-lg-up'}))
lista_paginacao

[[<div class="pagination-cell hidden-lg-up"><p>1 de 52</p></div>]]

#### Armazenando o numero de páginas em uma variável

In [113]:
for i in lista_paginacao:
    number = []
    number.append(i[0].find('p').get_text())
number = number[0].split()
number_page = int(number[2])
number_page

52

#### Coletando os dados via URL

In [114]:
url="https://www.adimimoveis.com.br/imoveis/a-venda"
html_list = []
for page in range(1, number_page + 1):
    lista = []
    req = Request(url + '?pagina=' + str(page), headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(req)
    bs = BeautifulSoup(html.read(), 'html.parser')
    lista.append(bs.find_all('div', {'class':'card_split_vertically__content'}))
    html_list.append(lista)

arq_html = open('imobiliario.html', 'w')
arq_html.write(str(html_list))
arq_html.close()

#### Selecionando os dados de interesse para análise dos valores dos imóveis em Balneário Camboriú

In [115]:
tabela = []
for dados in html_list:
    for cont in range(len(dados[0])):
        tabela.append(dados[0][cont])
tabela[0]

<div class="card_split_vertically__content"><div class="card_split_vertically__header"><p class="card_split_vertically__type">Apartamento</p><h3 class="card_split_vertically__location">Nações - Balneário Camboriú - SC</h3><ul class="card_split_vertically__specs"><li class="card_split_vertically__spec">64.91 m²</li><li class="card_split_vertically__spec">2 Quartos</li><li class="card_split_vertically__spec">1 Suíte</li><li class="card_split_vertically__spec">1 Banheiro</li><li class="card_split_vertically__spec">1 Vaga</li></ul></div><div class="card_split_vertically__footer"><div class="card_split_vertically__value-container"><p class="card_split_vertically__value-title">Venda</p><p class="card_split_vertically__value">R$ 786.433,15</p></div></div></div>

#### Coletando os textos de dentro das TAGS

In [116]:
new_table = []
for i in tabela:
    new_list = []
    new_list.append(i.find('p', {'class': 'card_split_vertically__type'}).text) # Tipo do imóvel
    new_list.append(i.find('h3', {'class': 'card_split_vertically__location'}).text) # Endereço
    li=i.find_all('li', {'class': 'card_split_vertically__spec'}) # m2, Quartos, Suítes e Vagas
    for l in range(len(li)):
        new_list.append(li[l].text) # m2, Quartos, Suítes e Vagas
    new_list.append(i.find('p', {'class':'card_split_vertically__value-title'}).text) # Tipo de tramite
    new_list.append(i.find('p', {'class':'card_split_vertically__value'}).text)
    new_table.append(new_list)  
new_table[:3]  

[['Apartamento',
  'Nações - Balneário Camboriú - SC',
  '64.91 m²',
  '2 Quartos',
  '1 Suíte',
  '1 Banheiro',
  '1 Vaga',
  'Venda',
  'R$ 786.433,15'],
 ['Apartamento',
  'Nações - Balneário Camboriú - SC',
  '98.83 m²',
  '2 Quartos',
  '2 Suítes',
  '1 Banheiro',
  '1 Vaga',
  'Venda',
  'R$ 907.063,86'],
 ['Apartamento',
  'Praia Brava - Itajaí - SC',
  '121.22 m²',
  '3 Quartos',
  '1 Suíte',
  '1 Banheiro',
  '2 Vagas',
  'Venda',
  'R$ 1.568.373,98']]

#### Criando o DataFrame

In [117]:
df = pd.DataFrame(new_table)
df.head(10)

,0,1,2,3,4,5,6,7,8
0,Apartamento,Nações - Balneário Camboriú - SC,64.91 m²,2 Quartos,1 Suíte,1 Banheiro,1 Vaga,Venda,"R$ 786.433,15"
1,Apartamento,Nações - Balneário Camboriú - SC,98.83 m²,2 Quartos,2 Suítes,1 Banheiro,1 Vaga,Venda,"R$ 907.063,86"
2,Apartamento,Praia Brava - Itajaí - SC,121.22 m²,3 Quartos,1 Suíte,1 Banheiro,2 Vagas,Venda,"R$ 1.568.373,98"
3,Apartamento,Praia Brava - Itajaí - SC,93.92 m²,2 Quartos,2 Suítes,1 Banheiro,1 Vaga,Venda,"R$ 1.139.371,54"
4,Apartamento,Nações - Balneário Camboriú - SC,62.4 m²,2 Quartos,1 Suíte,1 Banheiro,1 Vaga,Venda,"R$ 735.743,13"
5,Apartamento,Praia Brava - Itajaí - SC,93.92 m²,2 Quartos,2 Suítes,1 Banheiro,1 Vaga,Venda,"R$ 1.251.281,64"
6,Apartamento,Praia Brava - Itajaí - SC,121.22 m²,3 Quartos,1 Suíte,1 Banheiro,2 Vagas,Venda,"R$ 1.549.392,80"
7,Apartamento,Praia Brava - Itajaí - SC,93.92 m²,2 Quartos,2 Suítes,1 Banheiro,1 Vaga,Venda,"R$ 1.127.714,23"
8,Sala,Nações - Balneário Camboriú - SC,53.96 m²,1 Banheiro,Venda,"R$ 547.118,34",None,None,None
9,Sala,Nações - Balneário Camboriú - SC,54.93 m²,1 Banheiro,Venda,"R$ 558.638,42",None,None,None


#### Verificando o tamanho do DF

In [118]:
df.shape

(614, 9)

#### Analisando as informações iniciais dos dados colhidos

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       614 non-null    object
 1   1       614 non-null    object
 2   2       614 non-null    object
 3   3       614 non-null    object
 4   4       614 non-null    object
 5   5       608 non-null    object
 6   6       570 non-null    object
 7   7       539 non-null    object
 8   8       495 non-null    object
dtypes: object(9)
memory usage: 43.3+ KB


#### Tratando os valores nulos

In [120]:
df.isnull().sum()

0      0
1      0
2      0
3      0
4      0
5      6
6     44
7     75
8    119
dtype: int64

Eliminando os NaN da tabela

In [121]:
df.dropna(axis=0, inplace=True)
df.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64

In [122]:
df.fillna(df, inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8
0,Apartamento,Nações - Balneário Camboriú - SC,64.91 m²,2 Quartos,1 Suíte,1 Banheiro,1 Vaga,Venda,"R$ 786.433,15"
1,Apartamento,Nações - Balneário Camboriú - SC,98.83 m²,2 Quartos,2 Suítes,1 Banheiro,1 Vaga,Venda,"R$ 907.063,86"
2,Apartamento,Praia Brava - Itajaí - SC,121.22 m²,3 Quartos,1 Suíte,1 Banheiro,2 Vagas,Venda,"R$ 1.568.373,98"
3,Apartamento,Praia Brava - Itajaí - SC,93.92 m²,2 Quartos,2 Suítes,1 Banheiro,1 Vaga,Venda,"R$ 1.139.371,54"
4,Apartamento,Nações - Balneário Camboriú - SC,62.4 m²,2 Quartos,1 Suíte,1 Banheiro,1 Vaga,Venda,"R$ 735.743,13"


#### Iniciando o processo de limpeza das Strings 

In [123]:
df[8] = df[8].str.replace('[R$ ]', '')

/tmp/ipykernel_24117/1071970549.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df[8] = df[8].str.replace('[R$ ]', '')


In [124]:
df[2] = df[2].map(lambda x: x.split()[0])
df[3] = df[3].map(lambda x: x.split()[0])
df[4] = df[4].map(lambda x: x.split()[0])
df[5] = df[5].map(lambda x: x.split()[0])
df[6] = df[6].map(lambda x: x.split()[0])

In [125]:
new = df[1].str.split('-', expand=True)
df['bairro'] = new[0]
df['cidade'] = new[1]
df['estado'] = new[2]
df.drop(columns=[1], inplace=True)
df.head()

,0,2,3,4,5,6,7,8,bairro,cidade,estado
0,Apartamento,64.91,2,1,1,1,Venda,"786.433,15",Nações,Balneário Camboriú,SC
1,Apartamento,98.83,2,2,1,1,Venda,"907.063,86",Nações,Balneário Camboriú,SC
2,Apartamento,121.22,3,1,1,2,Venda,"1.568.373,98",Praia Brava,Itajaí,SC
3,Apartamento,93.92,2,2,1,1,Venda,"1.139.371,54",Praia Brava,Itajaí,SC
4,Apartamento,62.4,2,1,1,1,Venda,"735.743,13",Nações,Balneário Camboriú,SC


#### A coluna 7 refere-se ao contem valores do tipo de tranzação (vendas). Como todos tem o mesmo valor não precisamos desses dados, não terão peso em nossa analise.

In [126]:
df.drop(columns=[7], inplace=True)

#### Renomeando as colunas para facilitar o entendimento

In [127]:
df.rename(columns={0:'tipo_imovel', 2:'m2', 3:'quartos', 4:'suites', 5:'banheiros', 6:'vagas', 8:'preco'}, inplace=True)

#### Reorganizando a ordem das colunas para melhor assimilação

In [128]:
df = df[['cidade', 'bairro', 'estado', 'tipo_imovel', 'm2', 'quartos', 'suites', 'banheiros', 'vagas', 'preco']]
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Balneário Camboriú,Nações,SC,Apartamento,64.91,2,1,1,1,"786.433,15"
1,Balneário Camboriú,Nações,SC,Apartamento,98.83,2,2,1,1,"907.063,86"
2,Itajaí,Praia Brava,SC,Apartamento,121.22,3,1,1,2,"1.568.373,98"
3,Itajaí,Praia Brava,SC,Apartamento,93.92,2,2,1,1,"1.139.371,54"
4,Balneário Camboriú,Nações,SC,Apartamento,62.4,2,1,1,1,"735.743,13"


#### Convertendo as colunas de valores numéricos para int e float de acordo com seus critérios

In [130]:
df['m2'] = df['m2'].map(lambda x: float(x))
df['quartos'] = df['quartos'].map(lambda x: int(x))
df['suites'] = df['suites'].map(lambda x: int(x))
df['banheiros'] = df['banheiros'].map(lambda x: int(x))
df['vagas'] = df['vagas'].map(lambda x: int(x))

In [131]:
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Balneário Camboriú,Nações,SC,Apartamento,64.91,2,1,1,1,"786.433,15"
1,Balneário Camboriú,Nações,SC,Apartamento,98.83,2,2,1,1,"907.063,86"
2,Itajaí,Praia Brava,SC,Apartamento,121.22,3,1,1,2,"1.568.373,98"
3,Itajaí,Praia Brava,SC,Apartamento,93.92,2,2,1,1,"1.139.371,54"
4,Balneário Camboriú,Nações,SC,Apartamento,62.40,2,1,1,1,"735.743,13"


#### Limpando os dados da coluna "preco" pois nela encontram-se strings que impossibilitam a conversão para float.

In [132]:
df['preco'] = df['preco'].str.replace('[ .]', '')
df['preco'] = df['preco'].str.replace('[,]', '.')
df['preco'] = df['preco'].str.replace('Vendasobconsulta', '0')


/tmp/ipykernel_24117/7449221.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preco'] = df['preco'].str.replace('[ .]', '')
/tmp/ipykernel_24117/7449221.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preco'] = df['preco'].str.replace('[,]', '.')


In [133]:
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Balneário Camboriú,Nações,SC,Apartamento,64.91,2,1,1,1,786433.15
1,Balneário Camboriú,Nações,SC,Apartamento,98.83,2,2,1,1,907063.86
2,Itajaí,Praia Brava,SC,Apartamento,121.22,3,1,1,2,1568373.98
3,Itajaí,Praia Brava,SC,Apartamento,93.92,2,2,1,1,1139371.54
4,Balneário Camboriú,Nações,SC,Apartamento,62.40,2,1,1,1,735743.13


In [134]:
df['preco'] = df['preco'].map(lambda x: float(x))

In [135]:
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Balneário Camboriú,Nações,SC,Apartamento,64.91,2,1,1,1,786433.15
1,Balneário Camboriú,Nações,SC,Apartamento,98.83,2,2,1,1,907063.86
2,Itajaí,Praia Brava,SC,Apartamento,121.22,3,1,1,2,1568373.98
3,Itajaí,Praia Brava,SC,Apartamento,93.92,2,2,1,1,1139371.54
4,Balneário Camboriú,Nações,SC,Apartamento,62.40,2,1,1,1,735743.13


#### Verificando a quantia de cidades existentes na tabela

In [136]:
df['cidade'].unique()

array([' Balneário Camboriú ', ' Itajaí ', ' Camboriú ', ' Itapema '],
      dtype=object)

#### Conferindo a conversão dos tipos dos dados

In [140]:
df.dtypes

cidade          object
bairro          object
estado          object
tipo_imovel     object
m2             float64
quartos          int64
suites           int64
banheiros        int64
vagas            int64
preco          float64
dtype: object

#### Limpeza da Tabela Finalizada

In [142]:
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Balneário Camboriú,Nações,SC,Apartamento,64.91,2,1,1,1,786433.15
1,Balneário Camboriú,Nações,SC,Apartamento,98.83,2,2,1,1,907063.86
2,Itajaí,Praia Brava,SC,Apartamento,121.22,3,1,1,2,1568373.98
3,Itajaí,Praia Brava,SC,Apartamento,93.92,2,2,1,1,1139371.54
4,Balneário Camboriú,Nações,SC,Apartamento,62.40,2,1,1,1,735743.13


#### Salvando em um arquivo csv

In [143]:
df.to_csv('imobiliario.csv')